# TP - ES Problèmes inverses

Vincent Duval, Nicolas Desassis (à partir du TP en Matlab de Michel Kern)

Commencer par charger les différents packages nécessaires au TP.

In [ ]:
using Plots
using LinearAlgebra

## Exercice 1 (partie 1) - Méthodes de résolution de systèmes linéaires


### A) Utilisation des équations normales
La résolution d'un système linéaire $Ax=y$ au sens des moindres carrés, $$\min_x \frac{1}{2} \|Ax-y \|^2,$$ revient à résoudre les équations normales:
$$ A^\top (Ax-y)=0.$$

Dans toute cette page, on supposera que la matrice $A$ est **injective**, de sorte que la solution des moindres carrés est unique (égale à la solution fournie par le pseudo-inverse, $A^\dagger y$). 

On se propose d'utiliser les équations normales pour résoudre le système aux moindres carrés.


#### A.1) Factorisation LU
La méthode du pivot de Gauss permet de montrer le résultat suivant:

**Théorème:** <em>Soit $B\in \mathcal{M}_{n\times n}(\mathbb{R})$. Il existe des matrices $P, L, U \in \mathcal{M}_{n\times n}(\mathbb{R})$ telles $B=PLU$ avec:</em>
- <em>$P$ matrice de permutation,</em>
- <em>$L$ matrice triangulaire inférieure.</em>
- <em>$U$ une matrice triangulaire supérieure.</em>

<em>Si de plus les matrices extraites $B_k=(b_{i,j})_{1\leq i,j\leq k}$ sont inversibles pour tout $k\leq n$, alors on peut supposer $P=I$ et une telle décomposition est unique.
Le nombre d'opérations (additions, multiplications, divisions) pour la calculer est de l'ordre de $(A,M,D)\approx (n^3/3,n^3/3,n^2/2)$ opérations. On l'appelle **factorisation LU** de $B$.</em>

#### A.2) Méthode de Cholesky



**Théorème:** <em>Soit $B\in \mathcal{M}_{n\times n}(\mathbb{R})$ une matrice symétrique définie positive. Alors il existe une matrice triangulaire inférieure inversible $L\in \mathcal{M}_{n\times n}(\mathbb{R})$ telle que $B=L L^\top$. Cette factorisation s'appelle __factorisation de Cholesky__, on peut la calculer en $O(n^3)$ opérations. Plus précisément le nombre d'opérations (additions, multiplications, divisions) est de l'ordre de $(A,M,D)\approx (n^3/6,n^3/6, n^2/2)$ auquel s'ajoute $n$ extractions de racines carrées.</em>


1. Décrire sur le papier une méthode pour calculer la solution des équations normales avec les factorisations LU et de Cholesky.  A-t-on plutôt intérêt à utiliser la méthode LU ou de Cholesky?

2. Compléter le code suivant et essayer différentes valeurs de m et n ($m\geq n$)

In [ ]:
m,n= 15,14
# On crée des matrices et vecteurs au hasard
A=randn(m,n)
x=randn(n) # notre inconnue (la vraie solution)
y=A*x
# Résoudre les équations normales
B=A'*A

function solve_LU(B,z)
    Llu,Ulu=lu(B)
    return Ulu\(Llu\z)
end
# Remarque: cholesky et LU n'ont pas la même matrice L!
function solve_chol(B,z)
    Lc=cholesky(B).L
#    return  # COMPLETER

end
xlu=solve_LU(B,A'*y)
xchol=solve_chol(B,A'*y)
# Pour mesurer le temps
#@time xlu=solve_LU(B,A'*y)
#@time xchol=solve_chol(B,A'*y)


print("Erreur avec la décomposition LU: ")
println(norm(xlu-x)/norm(x))

print("Erreur avec la décomposition de Cholesky: ")
println(norm(xchol-x)/norm(x))

3. En fait, **former la matrice $A^\top A$ est instable numériquement**. 
Pour le comprendre on pourra comparer le conditionnement de $A^\top A$ avec celui de (la restriction) de A à son image.
Ou encore, on pourra considérer l'exemple suivant (cf  _Numerical methods for least squares problems_ de A. Bjorck)

$$ A = \begin{pmatrix}1 &1 &1\\ \varepsilon & 0& 0\\ 0 & \varepsilon &0 \\ 0 &0&\varepsilon \end{pmatrix} \quad A^\top A = \begin{pmatrix}1+\varepsilon^2 & 1 &1\\ 1 & 1+\varepsilon^2&1\\ 1 &1&1+\varepsilon^2 \end{pmatrix} $$
Si $\varepsilon=10^{-4}$, que dire de la matrice $A^\top A$?


### Factorisation QR

Nous allons proposer une méthode alternative, qui évite de former la matrice $A^\top A$.

**Théorème:** <em>Soit $A \in \mathcal{M}_{m\times n}(\mathbb{R})$ de rang $n$. Il existe une matrice orthogonale $Q\in \mathcal{M}_{m\times m}(\mathbb{R})$ et une matrice triangulaire $R\in \mathcal{M}_{n\times n}(\mathbb{R})$ inversible telles que $$ A = Q\begin{pmatrix}R\\0 \end{pmatrix}.$$
On l'appelle __factorisation QR__ de la matrive $A$. On peut la calculer en $(2mn^2 - 2/3n^3)$ opérations en virgule flottante.</em>



**Remarque:** Comparer avec la décomposition fournie par la méthode de Gram-Schmidt. En pratique ce n'est pas l'algorithme utilisé pour calculer la factorisation QR (elle est trop instable)!



**4.** En utilisant une décomposition QR de $A$, écrire le système vérifié par la solution $x$ des moindres carrés.

**5.** Compléter le code ci-dessous

In [ ]:
m,n= 15,14
# On crée des matrices et vecteurs au hasard
A=randn(m,n)
x=randn(n) # notre inconnue (la vraie solution)
y=A*x
# factorisation QR
Q,R=qr(A);  
#  xqr=   # COMPLETER
print("Erreur avec la décomposition QR: ")
println(norm(xqr-x)/norm(x))

## Exercice 1 (partie 2) - Application au lissage polynômial – stabilité des méthodes pour les moindres carrés

On cherche à approcher une fonction $f$ par un polynôme $x$ de degré donné. Il est connu que
l’interpolation conduit à des oscillations de grande amplitude, et l’on va plutôt minimiser
l’écart entre la fonction et le polynôme. Soient $t_i=\frac{i}{m-1}$, $i = 0,\dots, m-1$, les points où la fonction $f$
est connue. Notons $n$ le degré du polynôme, et $x_j$ , $j = 0,\dots, n$ ses coefficients.
Dans cet exemple, on prend $f (t) = \exp(\sin(4t))$ sur $[0, 1]$, avec $m = 100$ et $n = 14$.
Dans ce cas, la dernière composante de la solution $x$ « exacte » (calculée par Maple) vaut
2006.7874531048527. On divisera le second membre par cette valeur « magique », afin que
$x_{14} = 1$.

**1**. Montrer que la matrice et le second membre du problème de moindres carrés correspondant sont donnés par
$$A_{ij} = t_i^j,\hspace{5mm} y_i = f (t_i).$$
Ces formules se traduisent par le code Julia suivant :

In [ ]:
m=100
n=14

t = [ i/(m-1) for i in 0:(m-1)]
A = [tau^j for tau in t, j in 0:n]

magique=2006.7874531048527;
y=exp.(sin.(4*t))/magique;

**2**.  Comparer les solutions obtenues par les quatre méthodes suivantes :

- Equations normales

- Algorithme QR

- SVD de $A$

- Algorithme par défaut de Julia `x = A\y`

Que constatez-vous sur les résultats obtenus?

In [ ]:
x1=(A'*A)\(A'*y)

In [ ]:
Q,R = qr(A);
x2 = R\(Q[:,1:size(A,2)]'*y)

In [ ]:
U,S,V=svd(A);
x3 = V * ((1 ./S) .* (U'*y))

In [ ]:
x4 = A\y

**3.** Comparer graphiquement les données « expérimentales » avec les différentes prédictions
(remarquer qu’elles se calculent par `A*x`, où `x` est l’une des solutions calculées ci-dessus.

In [ ]:
plot(t, y,labels="Real")
plot!(t,A*x1,labels="Normal")
plot!(t,A*x2,labels="QR")
plot!(t,A*x3,labels="SVD")
plot!(t,A*x4,labels="Default")

**4.** Comment l'antislash "\\" (la méthode par défaut qui permet de résoudre les systèmes linéaires) fonctionne-t-elle?
On pourra consulter 
- la documentation de Julia
https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#Standard-functions

- ou le code source (lien issu de la documentation):
https://github.com/JuliaLang/julia/blob/bf534986350a991e4a1b29126de0342ffd76205e/stdlib/LinearAlgebra/src/generic.jl#L1097-L1127